# <a id='toc1_'></a>[The StructureData object](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [The StructureData object](#toc1_)    
  - [Initialization of a StructureData instance](#toc1_1_)    
    - [How to define the properties](#toc1_1_1_)    
    - [Inspect the supported and stored properties from the StructureData instance:](#toc1_1_2_)    
      - [Missing: access the supported property from the class object](#toc1_1_2_1_)    
    - [The immutability of the StructureData instance](#toc1_1_3_)    
    - [How to get kinds](#toc1_1_4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

Here we present the new `StructureData` class and we explain the basics on the API. 
This is an extended version of the `orm.StructureData` implemented in `aiida-core`.

Each property of the system is now stored in the `properties` attribute. Among the supported properties, we have:

- cell
- periodic boundary conditions (PBC)
- positions
- symbols 
- masses
- electronic charge
- magnetization - TOBE added
- Hubbard U and V parameters - TOBE added

Some of these properties are related to the sites/atoms (e.g. atomic positions, symbols, electronic charge) and some are related to the whole structure (e.g. PBC, cell). So, each property will have an attribute `domain`, which can be "intra-site", "inter-site", "global". 

<div style="border:2px solid #f7d117; padding: 10px; margin: 10px 0;">
    <strong>Important:</strong> we deprecate the kind-based definition of the properties, now site-based. This simplifies multiple properties defintion and respect the philosophy of a code-agnostic representation of the structure. The kinds determination can be done using the built-in `get_kinds` method of the StructureData. It is also possible to provide a user-defined set of kinds, as you will see in the following section.
</div>

The possibility to have user defined custom properties is discussed in another section (TOBE ADDED).

To explore the available properties in detail, please go to the corresponding pages (TOBE ADDED).

In [16]:
from aiida import orm, load_profile
load_profile()

Profile<uuid='827a33dde1bc4d498af8e022f9e9dece' name='mikibonacci'>

## <a id='toc1_1_'></a>[Initialization of a StructureData instance](#toc0_)
One of the principle of the new StructureData is the fact that it is "just" a container of the information about a given structure: this means that, after that instances of this class are immutable. After the initialization, it is not possible to change the stored properties.

### <a id='toc1_1_1_'></a>[How to define the properties](#toc0_)
Properties should be contained in a dictionary, where the keys are the names of the properties, and the values are dictionaries of this type:

```python
{
    "value":value_of_the_property,
    ... #other possible attributes of the property 
}
```

for example, we can provide the `positions` property by means of the following dictionary:
```python
{
    "value":[[1,1,1],[2,3,3],[3,4,5],[1,2,3]],
    "kind_tags":["H1","H2","H3","H4"],
}
```

where the `kind_tags` key provides user-defined set of kinds. This key is optional, as the kinds can (should) be determined at the plugin level.

Here below, an example in which we define a structure together with the mass and charge properties:

In [17]:
properties = {
    "cell":{"value":[[3.5, 0.0, 0.0], [0.0, 3.5, 0.0], [0.0, 0.0, 3.5]]},
    
    "pbc":{"value":[True,True,True]},
    
    "positions":{"value":[[0.0, 0.0, 0.0],[1.5, 1.5, 1.5]],},
    
    "symbols":{"value":["Li", "Li"]},
    
    "mass":{"value":[6.941,6.945],},
    
    "charge":{"value":[1,0]}
    }

In [18]:
from aiida_atomistic.data.structure.structure import StructureData

structure = StructureData(properties = properties)

We can inspect the properties by accessing the corresponding attribute (tab completion is enabled):

In [19]:
structure.properties.mass

Mass(parent=<StructureData: uuid: 72047b25-c462-4737-b06a-e4274afbc105 (unstored)>, value=[6.941, 6.945], domain='intra-site', default_kind_threshold=0.001)

In [20]:
structure.properties.mass.value

[6.941, 6.945]

In [21]:
structure.properties.mass.domain

'intra-site'

Additionally, we can access the property using the `get_property_attribute` method:

In [22]:
structure.properties.get_property_attribute("mass")

{'value': [6.941, 6.945]}

### <a id='toc1_1_2_'></a>[Inspect the supported and stored properties from the StructureData instance:](#toc0_)

In [23]:
structure.properties.get_supported_properties()

['pbc', 'mass', 'custom', 'cell', 'symbols', 'charge', 'positions']

In [24]:
structure.properties.get_stored_properties()

['mass', 'cell', 'pbc', 'symbols', 'charge', 'positions']

#### <a id='toc1_1_2_1_'></a>[Missing: access the supported property from the class object](#toc0_)

```python
In [1]: StructureData.get_supported_properties()
Out [2]: ['custom', 'pbc', 'symbols', 'charge', 'positions', 'mass', 'cell']
```

### <a id='toc1_1_3_'></a>[The immutability of the StructureData instance](#toc0_)

A crucial aspect of the new `StructureData` is that it is immutable even if the node is not stored, i.e. the API does not support on-the-fly or interactive modifications (it will raise errors). This helps in avoiding unexpected 
behaviour coming from a step-by-step defintion of the structure, e.g. incosistencies between properties definitions, which are then not cross-checked again.

One has to define a new `StructureData` instance by scratch.
To make user life simpler, we provide a `to_dict` method, which can be used to generate the properties dictionary. This can be updated and used for a new StructureData instance:

In [25]:
new_properties = structure.to_dict()
new_properties

{'cell': {'value': [[3.5, 0.0, 0.0], [0.0, 3.5, 0.0], [0.0, 0.0, 3.5]]},
 'pbc': {'value': [True, True, True]},
 'positions': {'value': [[0.0, 0.0, 0.0], [1.5, 1.5, 1.5]]},
 'symbols': {'value': ['Li', 'Li']},
 'mass': {'value': [6.941, 6.945]},
 'charge': {'value': [1, 0]}}

In [26]:
new_properties["mass"]["value"] = [6.941,6.941]

In [27]:
new_structure = StructureData(properties=new_properties)

In [28]:
new_structure.properties.mass.value

[6.941, 6.941]

### <a id='toc1_1_4_'></a>[How to get kinds](#toc0_)

It is possible to get a list of kinds using the `get_kinds` method. This will generate the corresponding predicted kinds for all the properties (the "intra-site" ones) and then generate the list of global different kinds. 


In [29]:
structure.get_kinds()

Li
Li


([0, 1],
 ['Li0', 'Li1'],
 {'mass': [6.9415, 6.945500000000001], 'charge': [1.1250000000000002, 0.225]})

As you can see, basically the value of a property is not the starting value choosen, and this is a PROBLEM: we have TOFIX this, especially in case in which the value is zero. 
The issue is that we are using a middle-point representative value.

Kinds are determined using, for each property, a given threshold. There is a default threshold:

In [30]:
structure.properties.mass.default_kind_threshold

0.001

Basically, it will be possible to override the threshold to be used for each property (TOBE implemented):

```python
structure.get_kinds(thr={"mass":0.1})
```